In [1]:
import yfinance as yf

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_ta as ta
from multiprocessing.pool import Pool
from sklearn.manifold import TSNE
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from collections import Counter
from sklearn.neural_network import MLPClassifier

In [3]:
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y= datasets.load_iris(return_X_y=True)
clf.fit(X, y)

import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X[0:1])
y[0]

0

In [5]:
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers.head())

  Symbol     Security             GICS Sector               GICS Sub-Industry  \
0    MMM           3M             Industrials        Industrial Conglomerates   
1    AOS  A. O. Smith             Industrials               Building Products   
2    ABT       Abbott             Health Care           Health Care Equipment   
3   ABBV       AbbVie             Health Care                   Biotechnology   
4    ACN    Accenture  Information Technology  IT Consulting & Other Services   

     Headquarters Location  Date added      CIK      Founded  
0    Saint Paul, Minnesota  1957-03-04    66740         1902  
1     Milwaukee, Wisconsin  2017-07-26    91142         1916  
2  North Chicago, Illinois  1957-03-04     1800         1888  
3  North Chicago, Illinois  2012-12-31  1551152  2013 (1888)  
4          Dublin, Ireland  2011-07-06  1467373         1989  


In [6]:
%%time
with open("hist_data_2018-1-1_2023-7-12.pkl","rb") as f:
    data = pickle.load(f)

CPU times: total: 0 ns
Wall time: 16.8 ms


In [7]:
columns = list(data["Open"].columns)

In [8]:
len(columns)

503

In [58]:
%%time
with open("hist_data_2018-1-1_2023-7-12_3_0_2.pkl","rb") as f:
    [training_X,training_Y,correlation_Y] = pickle.load(f)

CPU times: total: 0 ns
Wall time: 2.51 ms


In [59]:
training_X.shape,training_Y.shape,correlation_Y.shape

((66232, 14), (66232,), (66232,))

In [60]:
threshold = 0.02
train_data_y_discrete = np.asarray(list(map(lambda x: 2 if (x>threshold) else (1 if x<-threshold else 0),training_Y)))

# down sample

In [61]:
sel = np.hstack([np.where(train_data_y_discrete==0)[0][::10],np.where(train_data_y_discrete!=0)[0]])
downsamplespacing=5
training_X = training_X[sel][::downsamplespacing]
training_Y = training_Y[sel][::downsamplespacing]
correlation_Y = correlation_Y[sel][::downsamplespacing]

In [62]:
threshold = 0.02
train_data_y_discrete = np.asarray(list(map(lambda x: 2 if (x>threshold) else (1 if x<-threshold else 0),training_Y)))
Counter(train_data_y_discrete)

Counter({1: 1456, 2: 1204, 0: 1059})

In [63]:
X_train, X_test, y_train, y_test = train_test_split(training_X, train_data_y_discrete, test_size=0.3,random_state=109,shuffle=True)

In [64]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((2603, 14), (1116, 14), (2603,), (1116,))

In [65]:
%%time
# works clf = MLPClassifier(random_state=15, max_iter=500,hidden_layer_sizes=[150,150,50,10,50,150],).fit(X_train, y_train)
clf = MLPClassifier(random_state=15, max_iter=500,hidden_layer_sizes=[50,50,50,50],).fit(X_train, y_train)
y_pred = clf.predict(X_test)

CPU times: total: 984 ms
Wall time: 2.92 s


In [66]:
Counter(y_pred),Counter(y_test),Counter(y_train)

(Counter({1: 780, 2: 175, 0: 161}),
 Counter({1: 433, 2: 375, 0: 308}),
 Counter({1: 1023, 2: 829, 0: 751}))

In [67]:
print("out of samples results")
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred,average=None)) 
print("Recall:",metrics.recall_score(y_test, y_pred,average=None))

out of samples results
Accuracy: 0.40949820788530467
Precision: [0.32919255 0.42307692 0.42285714]
Recall: [0.17207792 0.76212471 0.19733333]


In [68]:
print("in samples results")
y_pred_train = clf.predict(X_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Precision:",metrics.precision_score(y_train, y_pred_train,average=None)) 
print("Recall:",metrics.recall_score(y_train, y_pred_train,average=None))

in samples results
Accuracy: 0.4114483288513254
Precision: [0.39952719 0.41586944 0.4044665 ]
Recall: [0.22503329 0.72238514 0.19662244]


In [70]:
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)

In [75]:
with open("3_0_3_test_save_model.clfpkl" ,"wb") as f:
    pickle.dump(clf,f)

In [76]:
with open("3_0_3_test_save_model.clfpkl" ,"rb") as f:
    s3 = pickle.load(f)

In [83]:
s3.coefs_

[array([[ 1.33511109e-01, -2.36869492e-03, -1.81270588e-02,
         -5.65457212e-02, -2.10223204e-01, -6.57541545e-02,
         -3.35357864e-02, -5.62615241e-02, -5.01616627e-02,
         -2.32349254e-04,  2.85739058e-02, -1.08326421e-01,
          4.20801122e-05,  6.33458897e-02,  1.65943691e-03,
         -2.69545019e-01, -1.77389389e-01, -2.40923845e-02,
         -2.20364608e-01, -2.18028693e-01,  3.25932111e-01,
         -2.66466077e-10,  1.55801778e-01,  1.36850786e-32,
          8.74481198e-03,  1.80069190e-02, -2.69737940e-01,
         -7.70326126e-02, -2.90644330e-01,  1.19009348e-02,
         -1.30669918e-05,  5.93983660e-02,  3.81374211e-07,
         -2.24743888e-09,  7.73978745e-11,  5.18153367e-02,
         -4.06713711e-22, -7.81051361e-02, -2.95820660e-02,
         -7.33428638e-03, -2.12093840e-01, -1.76343012e-02,
         -2.83780129e-01,  1.66911778e-01, -2.35714857e-02,
         -7.59409939e-03, -2.58748782e-01, -2.32046325e-02,
          1.26001622e-02, -5.64045596e-0

In [84]:
clf.coefs_

[array([[ 1.33511109e-01, -2.36869492e-03, -1.81270588e-02,
         -5.65457212e-02, -2.10223204e-01, -6.57541545e-02,
         -3.35357864e-02, -5.62615241e-02, -5.01616627e-02,
         -2.32349254e-04,  2.85739058e-02, -1.08326421e-01,
          4.20801122e-05,  6.33458897e-02,  1.65943691e-03,
         -2.69545019e-01, -1.77389389e-01, -2.40923845e-02,
         -2.20364608e-01, -2.18028693e-01,  3.25932111e-01,
         -2.66466077e-10,  1.55801778e-01,  1.36850786e-32,
          8.74481198e-03,  1.80069190e-02, -2.69737940e-01,
         -7.70326126e-02, -2.90644330e-01,  1.19009348e-02,
         -1.30669918e-05,  5.93983660e-02,  3.81374211e-07,
         -2.24743888e-09,  7.73978745e-11,  5.18153367e-02,
         -4.06713711e-22, -7.81051361e-02, -2.95820660e-02,
         -7.33428638e-03, -2.12093840e-01, -1.76343012e-02,
         -2.83780129e-01,  1.66911778e-01, -2.35714857e-02,
         -7.59409939e-03, -2.58748782e-01, -2.32046325e-02,
          1.26001622e-02, -5.64045596e-0